In [ ]:
# 主要步驟
#     讀取資料
#         讀檔案
#         取出常用變數(若有更改常用變數時要記得更新
#     清洗資料
#         刪除價格值為0的奇怪資料
#         刪除不會用到的欄位
#         刪除不會用到的列
#         找出資料誤值或空值
#         驗算資料
#     轉換資料
#         修正日期格式
#         以代號取代原資料(市場>>代號、花名重編、分類)
#         改成以時間為primary key 的大表格 (full_.csv)
#         以每年group by count 找出 前30名花種取交集刪減資料(reduce_.csv)
#         將上中下價分開(flat_.csv)
#     存取資料

In [4]:
# seting 讀取檔案
import os, glob
import numpy as np
import pandas as pd
fn_list = glob.glob("../raw_f_price/*.csv")
print(fn_list)
fn = fn_list[3]
print(fn)


['../raw_f_price\\allAnthurium.csv', '../raw_f_price\\allChrysanthemum.csv', '../raw_f_price\\allEustoma.csv', '../raw_f_price\\allOrientalLily.csv', '../raw_f_price\\allRose.csv']
../raw_f_price\allOrientalLily.csv


In [5]:
def trans_flat(fn):
    # 讀取檔案 先檢視檔案格式 將常用的變數先列出
    df = pd.read_csv(fn, encoding="utf-8")
    fn = fn.split("\\")[-1]
    print(fn)
    # 刪除不使用的資料 
    df = df.drop(["增減%","增減%.1","殘貨量"], axis=1)
    # 總行數 update
    nraw = df.shape[0]
    # 個別欄位名稱 update
    c = df.columns.tolist()
    # 資料檢查 part 1
    for colum in ['市場', '產品']:
        freq = df[colum].value_counts()
        dist_value = df[colum].drop_duplicates()
        null_data = df.iloc[df[colum].index.isnull()]

    # 將資料居然為 0 0 0 0的資料刪除
    flo_price = np.array(df["平均價"])
    # 將要刪除的index 放進去
    ind_to_delete=[]
    n = len(flo_price)
    for i in range(n):
        if int(flo_price[i])==0:
            ind_to_delete.append(i)
    # print("delete:", len(ind_to_delete))
    df = df.drop(ind_to_delete,axis=0)
    df.reset_index(drop=True)
    nraw = df.shape[0]

    # 刪除有問題的資料 (是小計的那行
    flo_name = np.array(df["產品"])
    # 將要刪除的index 放進去
    ind_to_delete=[]
    n = len(flo_name)
    for i in range(n):
        data = str(flo_name[i]).split(" ")
        if len(data)<3:
            ind_to_delete.append(i)
    df = df.drop(ind_to_delete,axis=0)
    df.reset_index(drop=True)
    nraw = df.shape[0]

    # 改變時間格式
    # 103/01/01 > 2014-01-01
    date_old = np.array(df["日期"])
    date_new = []
    dates = len(date_old)
    # dates = 10
    for i in range(dates):
        y,m,d = date_old[i].split("/")
        n_date = str(int(y)+1911) + "-" + m + "-" + d
        date_new.append(n_date)
    date_time_new = np.asarray(date_new)
    df["日期"] = date_time_new

    # 刪除 500 比筆以下資料 改每年前30名資料 篩選
    # drop_class=[]
    # for i in range(len(freq)):
    #     count = freq[i]
    #     item = freq.index[i]
    #     if count<500:
    #         drop_class.append(item)
    #         print(count, item)
    # drop_by_all = drop_class

    # 新稱欄位為year 做 group by
    d = np.array(df["日期"])
    year = []
    dates = len(d)
    for i in range(dates):
        date_data = d[i].split("-")
        year.append(date_data[0])
    df["year"] = year

    # 每年groyp by 取前30名
    gdf  = df.groupby("year")
    year = ["2014", "2015", "2016", "2017", "2018"]
    df_by_year=[]
    year_rank_30=[]
    for y in year:
        t_df = gdf.get_group(y)
        freq = t_df["產品"].value_counts()
        dist_value = df[colum].drop_duplicates()
        # print(freq)
        # print(dist_value)
        r30 = set()
        for i in range(30):
            count = freq[i]
            item = freq.index[i]
    #         print(i,count, item)
            r30.add(item)
        year_rank_30.append(r30)

    # 取各年交集
    for i in range(len(year_rank_30)):
        if i==0:
            all_rank30 = year_rank_30[i]
        all_rank30 = all_rank30.intersection(year_rank_30[i])
    # print(len(all_rank30),list(all_rank30))
    finall_class = list(all_rank30)

    df = df.drop(["year"],axis=1)

    # 刪資料前存檔
    fn_full = "../result/" + "full_" + fn
    df.to_csv(fn_full, index=False, encoding="utf-8" )
    print(fn_full, "saved")
    # 取出finall_class 的資料
    df_arr = np.array(df)
    n = len(df_arr)
    index_in_final=[]
    for i in range(n):
        if df_arr[i,2] in finall_class:
    #         print(i, df_arr[i,2])
            index_in_final.append(i)

    df_arr = df_arr[index_in_final,:]
    df_new = pd.DataFrame(df_arr, columns=c)

    # 存檔 以刪減一次資料
    fn_new = "../result/" + "reduce_"+fn
    df_new.to_csv(fn_new, index=False, encoding="utf-8" )
    print(fn_new, "saved")
    # 攤平資料
    c_new = ["日期", "市場", "產品", "價別", "價格", "交易量"]
    # 1 row to 3 new row 
    flat_data =[]
    for i in range(len(df_arr)):
        o_d = df_arr[i,:]
    #     print(o_d)
        l = list(o_d[0:3])
    #     print(l)
        p_h = l + ["上價"] +  [int(o_d[4])] + [round(o_d[8]*0.2)]
        p_m = l + ["中價"] +  [int(o_d[5])] + [round(o_d[8]*0.4)]
        p_l = l + ["下價"] +  [int(o_d[6])] + [round(o_d[8]*0.2)]
        flat_data.append(p_h)
        flat_data.append(p_m)
        flat_data.append(p_l)
    df_flat = pd.DataFrame(flat_data, columns=c_new)

    # 儲存攤平資料
    fn_flat = "../result/" + "flat_"+fn
    df_flat.to_csv(fn_flat, index=False, encoding="utf-8" )
    print(fn_flat, "saved")

In [6]:
# main
fn_list = glob.glob("../raw_f_price/*.csv")
# for fn in fn_list:
#     trans_flat(fn)
trans_flat(fn)

allOrientalLily.csv
../result/full_allOrientalLily.csv saved
../result/reduce_allOrientalLily.csv saved
../result/flat_allOrientalLily.csv saved
